In [9]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import time
import copy



class VSURF:
    def __init__(self, num_var,ytype ='continuous' ,n_estimators = 2000):
        self.ytype = ytype
        self.mtry = max(int(num_var / 3), 1) # refered by VSURF paper
        self.n_estimators = n_estimators


    def fit(self, X_train, y_train, num_runs=50):
        variable_importances = []
        for i in range(num_runs):
            if self.ytype == 'continuous':
                regression_model = RandomForestRegressor(n_estimators=self.n_estimators, 
                                                         max_features = self.mtry  ,random_state=i,n_jobs = -1)
                regression_model.fit(X_train, y_train)
                variable_importances.append(regression_model.feature_importances_)
            else:
                classfication_model = RandomForestClassifier(n_estimators=self.n_estimators,
                                                             max_features = self.mtry ,random_state = i, n_jobs = -1)
                classfication_model(X_train, y_train)
                variable_importances.append(classfication_model.feature_importances_)
            
        vi_mean = np.mean(variable_importances, axis=0)
        vi_std = np.std(variable_importances, axis=0)
        self.vi_mean = copy.deepcopy(vi_mean)

        sorted_indices = np.argsort(vi_std)[::-1]
        
        
        sorted_data = sorted(range(len(vi_mean)), key=lambda x: vi_mean[x], reverse=True)  # 데이터를 기준으로 내림차순으로 인덱스 정렬
        ranked_data = {index: rank + 1 for rank, index in enumerate(sorted_data)}  # 인덱스에 등수 부여

        xranks = [ranked_data[index] for index in range(len(vi_mean))]  # 각 값의 인덱스에 해당하는 등수
        X_ranks = np.array(xranks).reshape(-1, 1)
        
        max_depths = range(1, 20)
        cp_values = []
        
        # prunning
        for max_depth in max_depths:
            cart_model = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
            cart_model.fit(X_ranks, vi_std)
            
            y_pred = cart_model.predict(X_ranks)
            sse_p = np.sum((y_pred - vi_std)**2)
            
            mean_std = [np.mean(vi_std)] * len(vi_std)
            
            mse_full = mean_squared_error(vi_std, mean_std)
            n = len(vi_std)
            d_p = cart_model.get_n_leaves()
            cp = (sse_p / mse_full) - (n - 2 * d_p)
            cp_values.append(cp)
        
        print('cp_values:', cp_values)
        best_max_depth = max_depths[np.argmin(cp_values)]
        final_model = DecisionTreeRegressor(max_depth=best_max_depth, random_state=42)
        final_model.fit(X_ranks, vi_std)

        self.threshold = np.min(final_model.predict(X_ranks))
        print('thereshold:', self.threshold)
        print("VI mean", vi_mean)
        print("VI STD:", vi_std)

        self.selected_variables = [idx for idx in sorted_indices if vi_mean[idx] > self.threshold]

    def get_selected_variables(self):
        return self.selected_variables
    
    # def plot_variable_importance(self):
        # graph = sns.lineplot(data= self.vi_mean, x="VI mean", y="Variable")
        # graph.axhline(self.threshold, c = 'red')
        # plt.show()
            

# Simulation study

start_time = time.time()

data = fetch_california_housing()

num_noise_features = 50

noise = np.random.normal(0, 1, (data.data.shape[0], num_noise_features))

data_with_noise = np.hstack((data.data, noise))

X_train, y_train = data.data, data.target


# 클래스 객체 생성 및 변수 선택 수행
variable_selector = VSURF(num_var = X_train.shape[1], ytype = 'continuous')
variable_selector.fit(X_train, y_train)

# 선택된 변수들의 인덱스 출력 (VI 값이 높은 순서대로)
print("Selected Variables (Ranked by VI):", variable_selector.get_selected_variables())

end_time = time.time()

elapsed_time_in_seconds = end_time - start_time

elapsed_time_in_minutes = elapsed_time_in_seconds / 60

print(f"Elapsed Time: {elapsed_time_in_minutes:.2f} minutes")

cp_values: [-1.5124191141591838, -1.191442072807072, 2.3600425748602896, 6.000039473867371, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0]
thereshold: 0.0005560265467308789
VI mean [0.35466521 0.05604506 0.11390345 0.04402107 0.0351941  0.12188809
 0.13866434 0.13561867]
VI STD: [2.13202980e-03 2.18204148e-04 1.16234860e-03 3.06667874e-04
 7.79370152e-05 4.05671266e-04 8.57888941e-04 8.63467986e-04]
Selected Variables (Ranked by VI): [0, 2, 7, 6, 5, 3, 1, 4]
Elapsed Time: 7.57 minutes


In [10]:
len(variable_selector.get_selected_variables())

8

In [ ]:
variable_selector.plot_variable_importance()